In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from summarizer import TransformerSummarizer

In [ ]:
!pip install bert-extractive-summarizer
!pip install transformers==2.2.0
!pip install spacy==2.0.12

In [ ]:
data = pd.read_csv("competitions_info.csv")

In [ ]:
data.head()

,ref,comp_name,comp_type,Description,Metric,DataType,Subject,ProblemType
0,c/essay-scoring,Essay Scoring,inClass,<p>This contest is a clone of the short-answer...,custom metric,NaN,NaN,NaN
1,c/Clustering,BU CS 565 : Clustering (1/3),inClass,"<p>In this project, you are given a set of 100...",rmse,NaN,NaN,NaN
2,c/us-census-challenge,U.S. Census Return Rate Challenge,featured,<p><strong>Note: The prediction phase of this ...,nwmae,NaN,NaN,NaN
3,c/cs6998,CS6998 Homework 2,inClass,<p>homework 2:&nbsp;Text Classification</p>,auc,NaN,NaN,NaN
4,c/buadm12,BUADM12,inClass,"<div class=""x_x_x_x_x_x_x_x_x_x_moz-text-flowe...",custom metric,NaN,NaN,NaN


## Удалим ссылки и html разметку

In [ ]:
def preprocess(text):
    text = str(text)
    text = text.replace("\n", " ")
    return text

In [ ]:
docs = []

for doc in data.Description.values:
    parsed_html = BeautifulSoup(str(doc))
    for script in parsed_html(["script", "style"]):
        script.decompose()
    docs.append(" ".join(list(parsed_html.stripped_strings)).replace("\n", " "))

## Применим BART трансформер для выделения темы описания для текстов, длиннее 1500 символов

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['My friends']


In [ ]:
summarized_docs = []
changed_flag = []

for doc in tqdm(docs):
    if len(doc) > 1500:
        try:
            inputs = tokenizer([doc], return_tensors='pt')
            summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True)
            summarized_text = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
            print(summarized_text)
            summarized_docs.append(summarized_text)
            changed_flag.append(True)
        except IndexError:
            summarized_docs.append(doc)
            changed_flag.append(False)
    else:
        summarized_docs.append(doc)
        changed_flag.append(False)



  0%|          | 0/5060 [00:00<?, ?it/s]

  0%|          | 6/5060 [00:38<8:55:42,  6.36s/it]

There are 5 essay sets. Each of the sets of essays was generated from a single prompt. Selected essays range from an average length of 150 to 550 words per response. All responses were written by students ranging in grade levels from Grade 7 to Grade 10. All essays were hand graded and were double-scored.




  0%|          | 9/5060 [01:22<12:29:40,  8.91s/it]

Great Candidates of America (GCA) is one of the largest direct mail fund-raisers in the country. Your objective is to demonstrate the performance of their tool by analyzing the results of one of GCA's fund raising appeals. This mailing was sent to a total of 3.5 million GCA donors who were on the GCA database as of June 1997.




  0%|          | 13/5060 [02:03<13:04:43,  9.33s/it]

...

There are 4 datasets (each will be released at specified times) There is the test data set that you need to prepare the results for. This data is proprietary please do not share the dataset with anyone. Use any model taught in class or otherwise and race to the leader-board! Good luck! ---Sayan---




100%|█████████▉| 5043/5060 [6:10:40<03:01, 10.69s/it]

Keditausfallprognose - Loan Default Prediction (LDP) Ein Finanzinstitut macht mit Kreditausfällen bei Fahrzeugkrediten große Verluste. Deshalb möchte das Finanzinsitut die bestehenden Modelle für die Prognose von Krediten mit Modellen ersetzten, die die Ausfallwahrscheinlichkeiten treffsicherer vorhersagen.




100%|██████████| 5060/5060 [6:11:13<00:00,  4.40s/it]

As part of the Kainos AI Camp, we are running our own (small) Kaggle competition. The aim is to train a classifier that will label each image correctly. You can have a team of up to three people. The top two submissions (as in, the two best submissions) from each team will be scored.


## Сохраним выделенные тексты

In [ ]:
data["Summarized"] = summarized_docs
data["Summarized_flag"] = changed_flag

In [ ]:
data.to_csv("competitions_info_summ_bart.csv")

In [ ]:
from google.colab import files

files.download("competitions_info_summ_bart.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Применим GPT2 трансформер для выделения темы описания для текстов, длиннее 1500 символов


In [ ]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")
# full = ''.join(GPT2_model(body, min_length=60))
# print(full)

100%|██████████| 456318/456318 [00:00<00:00, 1062321.85B/s]


In [ ]:
summarized_docs = []
changed_flag = []

for doc in tqdm(docs):
    if len(doc) > 1500:
        summ_text = ''.join(GPT2_model(doc, min_length=60))
        print(summ_text)
        summarized_docs.append(summ_text)
        changed_flag.append(True)
    else:
        summarized_docs.append(doc)
        changed_flag.append(False)

  0%|          | 6/5060 [00:08<2:02:14,  1.45s/it]

Your success depends upon how closely you can deliver scores to those of human expert graders. The variability is intended to test the  limits of your scoring engine's capabilities. If applicable, the Readme file also includes the source information for essays that required students to  read and respond to an excerpt. But if you talk about @PERSON3, I can't tell that's the same person." "...


  0%|          | 9/5060 [00:23<3:27:59,  2.47s/it]

BACKGROUND AND OBJECTIVES The data set for this project has been generously provided by a not-for-profit organization,  with an in-house database of over 13 million donors. They represent an important group to GCA, since the longer someone goes without donating, the less likely  they will be to give again. Therefore, recapture of these former donors is a critical aspect of GCA's fund raising efforts. The analysis file includes all 95,412 Lapsed donors who received the mailing, with responders to the mailing marked with a flag in the TARGET_B field. The overall response rate for this direct mail promotion is around 5%. The package cost (including the mail cost) is $0.85 per piece mailed. (


  0%|          | 12/5060 [00:45<5:34:35,  3.98s/it]

...

Keditausfallprognose - Loan Default Prediction (LDP) Ein Finanzinstitut macht mit Kreditausfällen bei Fahrzeugkrediten große Verluste. die Antragstellerin selbst für sich als Hauptantragsteller/in abgeschlossen und bei sekundären Kreditverträgen (secondary, kurz: sec) ist der Antragsteller bzw. Sichtbar sind jedoch nur die Bewertung Ihrer Ergebnisse einer Stichprobe, damit Sie die Bewertung der Einreichung nicht zur Optimierung Ihres Modells verwenden können.


100%|█████████▉| 5049/5060 [2:24:35<00:38,  3.49s/it]

As part of the Kainos AI Camp, we are running our own (small) Kaggle competition. These are: airplane, car, cat, dog, flower, fruit, motorbike and person. The scoring will happen on a part of the dataset that you are not given to train on, so make sure your validation accuracy is high! ### Advice  Convolutional Neural Nets are much better at classifying images than other kinds of Neural Nets. ### Prizes  Every member of the winning team will get a mystery prize, which is being awarded at the end of the Hackathon on Saturday.


100%|██████████| 5060/5060 [2:24:54<00:00,  1.72s/it]

ISFOG 2020 - Pile driving prediction event INTRODUCTION Data science techniques are rapidly transforming businesses in a broad range of sectors. In the context of the ISFOG2020 conference in Austin, TX, a prediction event is launched which invites geotechnical engineers to share knowledge and gain hands-on experience with machine learning models. In current engineering practice, blowcounts are derived using a three-step approach: Predict the soil resistance vs depth from geotechnical data (Alm and Hamre, 2001; Stevens et al., GUIDELINES FOR PARTICIPATION Target audience This prediction event targets individuals or groups with an interest in data science techniques. The final scores based on the full test dataset will be revealed once the competition ends. Presented at the International Conference On Soil Mechanics and Foundation Engineering. Acknowledgements We thank Cathie Group for providing this dataset.


In [ ]:

data["Summarized"] = summarized_docs
data["Summarized_flag"] = changed_flag

In [ ]:
data.to_csv("competitions_info_summ_gpt2.csv")

In [ ]:
from google.colab import files

files.download("competitions_info_summ_gpt2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
bart_res = data[["Description", "Summarized", "Summarized_flag"]]
bart_res["Preprocessed"] = docs

bart_res.to_csv("bart_summ.csv")
files.download("bart_summ.csv")